# MFCCS
Obtener los mfccs para 5 clases c/u de 500 audios, para un total de 15.000 audios

In [12]:
import librosa
import IPython.display as ipd
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import sklearn as sk
import pandas as pd
from sklearn import preprocessing

In [264]:
#PATHSAMPLES = '../data/training_list_test_temporal.txt'
PATHSAMPLES = '../data/training_list.txt'
NSAMPLES = 30
SOUNDS =np.array(['yes','no','right','five','nine'])
#SOUNDS =np.array(['off','five','left'])

In [265]:
def get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS):
    ROOTPATH= 'data/'
    data = pd.read_csv(PATHSAMPLES, sep=" ", header=None)
    data = data.values #Convertir en numpy array
    labels = []
    speakers  = []
    paths = []
    for path in data:
        pathSplit =path[0].split('/')
        speakerName = pathSplit[1].split('_')[0]
        word = pathSplit[0]
        isWord = np.where(SOUNDS == word)
        if len(isWord[0]) != 0:
            label = SOUNDS[isWord[0][0]]
            labels.append(label)
            speakers.append(speakerName)
            paths.append(ROOTPATH+path[0])
    NSAMPLES = NSAMPLES
    first = labels.index(SOUNDS[0])
    last = first +NSAMPLES
    matrixMajor = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
    for i in range(1,np.size(SOUNDS,0)):
        first = labels.index(SOUNDS[i])
        last = first +NSAMPLES
        clasei = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
        matrixMajor = np.append(matrixMajor,clasei,axis=0)
    return matrixMajor

In [266]:
path_word_speakerId = get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS)

In [267]:
path_word_speakerId.shape

(150, 3)

In [268]:
path_word_speakerId

array([['data/yes/004ae714_nohash_0.wav', 'yes', '004ae714'],
       ['data/yes/004ae714_nohash_1.wav', 'yes', '004ae714'],
       ['data/yes/00970ce1_nohash_0.wav', 'yes', '00970ce1'],
       ['data/yes/00f0204f_nohash_0.wav', 'yes', '00f0204f'],
       ['data/yes/00f0204f_nohash_1.wav', 'yes', '00f0204f'],
       ['data/yes/00f0204f_nohash_2.wav', 'yes', '00f0204f'],
       ['data/yes/012c8314_nohash_0.wav', 'yes', '012c8314'],
       ['data/yes/0132a06d_nohash_0.wav', 'yes', '0132a06d'],
       ['data/yes/0132a06d_nohash_1.wav', 'yes', '0132a06d'],
       ['data/yes/0132a06d_nohash_2.wav', 'yes', '0132a06d'],
       ['data/yes/0132a06d_nohash_3.wav', 'yes', '0132a06d'],
       ['data/yes/0132a06d_nohash_4.wav', 'yes', '0132a06d'],
       ['data/yes/0135f3f2_nohash_0.wav', 'yes', '0135f3f2'],
       ['data/yes/0135f3f2_nohash_1.wav', 'yes', '0135f3f2'],
       ['data/yes/0137b3f4_nohash_0.wav', 'yes', '0137b3f4'],
       ['data/yes/0137b3f4_nohash_1.wav', 'yes', '0137b3f4'],
       [

In [269]:
np.unique(path_word_speakerId[:,2])

array(['004ae714', '00970ce1', '00b01445', '00f0204f', '012187a4',
       '012c8314', '0132a06d', '0135f3f2', '0137b3f4', '014f9f65',
       '01648c51', '0165e0e8', '016e2c6d', '017c4098', '019fa366',
       '01bb6a2a', '01bcfc0c', '01d22d03', '0227998e', '023808be',
       '023a61ad', '02746d24', '02ade946'], dtype='<U32')

In [270]:
def build_features(path_word_speakerId, n_mfcc,NSAMPLES, PATHSAMPLES, SOUNDS):
    N_rows = NSAMPLES*len(SOUNDS)*n_mfcc
    N_colums = n_mfcc +3# número de columnas del mfcc + 3 columnas (la palabra y el speaker id, path al que pertenece)
    # el path es necesario ya que estamos teniendo un enfoque de multiples instancias.
    path_word_speakerId = get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS)
    matrix_features = np.empty((1,23)) # inicialmente

    #Encode etiqueta del audio,speaker y la palabra
    lePath = preprocessing.LabelEncoder()
    leSpeaker = preprocessing.LabelEncoder()
    leWord =  preprocessing.LabelEncoder()
    
    lePath.fit(path_word_speakerId[:,0])
    leSpeaker.fit(path_word_speakerId[:,2])
    leWord.fit(path_word_speakerId[:,1])
    
        
    for sound_info in path_word_speakerId:
        audio_path = '../'+sound_info[0]
        x , sr = librosa.load(audio_path)
        x_normalize=sk.preprocessing.minmax_scale(x, axis=0)       
        mfccs = librosa.feature.mfcc(x_normalize, sr=sr,n_mfcc=n_mfcc,hop_length=int(0.010*sr), n_fft=int(0.025*sr))
        n_windows = mfccs.shape[1]
        path = sound_info[0]
        speaker_id = sound_info[2]
        word=sound_info[1]        
        labelPath = lePath.transform([path])
        labelSpeaker = leSpeaker.transform([speaker_id])
        labelWord= leWord.transform([word])                
        labelPath = np.repeat(labelPath,10)
        labelWord =  np.repeat(labelWord,10)
        labelSpeaker= np.repeat(labelSpeaker,10)        
        fila_P_W_S = np.array([labelPath,labelWord,labelSpeaker]).T  
        #mfccFull =  np.hstack((mfccs.T,fila_P_W_S)) 
        #Selecciona de 10 y se promedian
        mfccFull_split = np.array_split(mfccs.T,10)
        avg_splis = np.ones((10,20))
        i = 0
        for spli in mfccFull_split:
            avg_spli = np.average(spli,axis=0)            
            avg_splis[i]=avg_spli
            i = i+1
        mfccFull =  np.hstack((avg_splis,fila_P_W_S))
        
        matrix_features = np.concatenate((matrix_features,mfccFull), axis=0)        
    return matrix_features[1:,:]

In [271]:
matrixFinal = build_features(path_word_speakerId, 20,NSAMPLES, PATHSAMPLES, SOUNDS)
print(matrixFinal.shape)

(1500, 23)


In [272]:
np.savetxt('../data/audios_MFCC_average.csv',matrixFinal,delimiter=",")

In [273]:
data_avg = pd.read_csv('../data/audios_MFCC_average.csv', sep=",", header=None)
data_avg = data_avg.values #Convertir en numpy array
data_avg.shape

(1500, 23)